In [3]:
import numpy as np
from scipy.stats import norm


def self_learning_algorithm(true_means: np.ndarray, true_probabilities: np.ndarray, data_size: int, data: np.ndarray, tolerance: float = 1e-3, max_iterations: int = 20000) -> tuple[np.ndarray, np.ndarray]:
    """
    Self-learning algorithm to estimate mixture model parameters using Expectation-Maximization (EM).

    Args:
        true_means (np.ndarray): True means of the components.
        true_probabilities (np.ndarray): True probabilities of the components.
        data_size (int): Number of data points.
        data (np.ndarray): Observed data.
        tolerance (float, optional): Convergence tolerance for parameter updates. Defaults to 1e-3.
        max_iterations (int, optional): Maximum number of iterations. Defaults to 20,000.

    Returns:
        tuple[np.ndarray, np.ndarray]: Estimated probabilities and means of the components.
    """
    print("=" * 100)
    print(f"Starting Self-Learning Algorithm\nData size: {data_size}\n")

    # Initialize means and probabilities
    estimated_means = np.linspace(min(data), max(data), 4)
    estimated_probabilities  = np.ones(4) / 4

    print(f"Initial estimated means: {estimated_means}")
    print(f"Initial estimated probabilities: {estimated_probabilities }")

    for iteration in range(max_iterations):
        # E-Step: Calculate responsibilities (gamma)
        responsibilities = np.zeros((data_size, 4))
        for k in range(4):
            responsibilities[:, k] = estimated_probabilities[k] * norm.pdf(data, loc=estimated_means[k], scale=1)
        responsibilities /= np.maximum(responsibilities.sum(axis=1, keepdims=True), 1e-10)

        # M-Step: Update means and probabilities
        new_means = np.sum(responsibilities * data[:, np.newaxis], axis=0) / (responsibilities.sum(axis=0) + 1e-10)
        new_probabilities = responsibilities.sum(axis=0) / data_size

        # Check convergence
        if np.all(np.abs(new_probabilities - estimated_probabilities) < 1e-5) and np.all(np.abs(new_means - estimated_means) < tolerance):
            print(f"Convergence reached after {iteration + 1} iterations.")
            break

        estimated_means, estimated_probabilities = new_means, new_probabilities

    print(f"Algorithm completed in {iteration + 1} iterations.")
    print(f"True means: {true_means}")
    print(f"True probabilities: {true_probabilities}")
    print("\nEstimated parameters:")
    print(f"Means: {estimated_means.round(2)}")
    print(f"Probabilities: {estimated_probabilities.round(2)}")
    print("\nDifferences:")
    print(f"Means difference: {np.abs(true_means - estimated_means)}")
    print(f"Probabilities difference: {np.abs(true_probabilities - estimated_probabilities)}\n")

    return estimated_probabilities , estimated_means


sample_sizes = [10, 100, 1000, 5000, 10000]
true_means = np.array([0, 1, 2, 3])
true_probabilities = np.array([(k + 1) / 10 for k in true_means])
true_probabilities /= true_probabilities.sum()

# Run the algorithm for different data sizes
for size in sample_sizes:
    print(f"\nGenerating {size} samples...\n")

    # Generate sample data
    sampled_means = np.random.choice(true_means, size=size, p=true_probabilities)
    generated_data = np.random.normal(loc=sampled_means, scale=1, size=size)

    # Apply the self-learning algorithm
    final_probabilities, final_means = self_learning_algorithm(true_means=true_means, true_probabilities=true_probabilities, data_size=size, data=generated_data)


Generating 10 samples...

Starting Self-Learning Algorithm
Data size: 10

Initial estimated means: [-0.17220465  0.92623592  2.02467649  3.12311707]
Initial estimated probabilities: [0.25 0.25 0.25 0.25]
Convergence reached after 78 iterations.
Algorithm completed in 78 iterations.
True means: [0 1 2 3]
True probabilities: [0.1 0.2 0.3 0.4]

Estimated parameters:
Means: [0.61 2.15 2.15 2.15]
Probabilities: [0.13 0.19 0.37 0.31]

Differences:
Means difference: [0.61174247 1.14556935 0.14556953 0.85443047]
Probabilities difference: [0.02888341 0.01192288 0.06948261 0.08644314]


Generating 100 samples...

Starting Self-Learning Algorithm
Data size: 100

Initial estimated means: [-1.2103676   1.04867791  3.30772342  5.56676893]
Initial estimated probabilities: [0.25 0.25 0.25 0.25]
Convergence reached after 238 iterations.
Algorithm completed in 238 iterations.
True means: [0 1 2 3]
True probabilities: [0.1 0.2 0.3 0.4]

Estimated parameters:
Means: [0.62 0.62 2.2  4.48]
Probabilities: [

Алгоритм показує значну залежність від кількості даних n та початкових оцінок параметрів. Зі збільшенням обсягу даних n, його результати стають більш точними та стабільними. Для невеликих значень n (наприклад, n = 10) алгоритм швидко збігається, але результати мають високу варіативність і значні відхилення від істинних значень через обмеженість спостережень. Середні значення та ймовірності для таких обсягів даних оцінюються ненадійно.

Для середніх значень n (наприклад, n = 100) точність зростає, але алгоритм все ще демонструє певні похибки, особливо у визначенні ймовірностей. Великі n (наприклад, n = 5000 або n = 10000) дозволяють алгоритму значно зменшити похибки оцінок середніх і ймовірностей. Водночас зростає кількість ітерацій, необхідних для досягнення зближення, оскільки більший обсяг даних потребує більше обчислень для оновлення параметрів.

Початкові оцінки параметрів також впливають на швидкість і точність алгоритму. Якщо початкові значення середніх mu_k або ймовірностей p_k значно відрізняються від істинних, алгоритму потрібно більше ітерацій для зближення. Проте навіть при далеких від істини початкових оцінках результати можуть бути задовільними за великого n, адже великий набір даних компенсує невдалий старт. Однак при малих значеннях n невдалий вибір початкових параметрів значно ускладнює отримання точних оцінок.

Час зближення залежить від обсягу даних. Для невеликих n алгоритм збігається швидше, оскільки набір даних малий і зміни параметрів на кожній ітерації значні. Для великих n алгоритм потребує більше ітерацій через те, що обсяг даних уповільнює адаптацію параметрів.

Щодо оцінок параметрів, середні mu_k відновлюються точніше, ніж ймовірності p_k. Особливо добре оцінюються середні для компонентів із високими ймовірностями, наприклад, mu_3 при p_3 = 0.4. Натомість алгоритм має труднощі з правильною оцінкою малих ймовірностей, таких як p_0 = 0.1, навіть за великого обсягу даних.

Загалом, алгоритм демонструє хорошу поведінку для великих обсягів даних (n >= 1000), оскільки статистична репрезентативність вибірки допомагає отримати точні оцінки параметрів. Проте при малих обсягах даних або неправильних початкових оцінках результати можуть бути неточними, особливо для ймовірностей. Для підвищення ефективності бажано використовувати достатньо великі n, а початкові оцінки слід обирати якомога ближче до очікуваних значень. Це дозволить прискорити зближення алгоритму та підвищити точність його результатів.